In [1]:
import sys, os
import torch
import torch.nn as nn
import math
import numpy as np

In [89]:
class LayerPerceptron(torch.nn.Module):
    def __init__(
        self,
        in_dim     = None,
        out_dim    = None,
        w_init     = False,
        b_init     = False,
        act        = nn.Tanh(),
    ):
        """
            Initialize LayerPerceptron
        """
        super(LayerPerceptron,self).__init__()
        self.dense      = nn.Linear(in_features=in_dim,
                                    out_features=out_dim,
                                    dtype=torch.float)

        self.activation = act
        # Initialize parameters
        self.init_param(w_init, b_init)

    def init_param(self, w_init, b_init):
        """
            Initialize parameters
        """
        if w_init:
            nn.init.constant_(self.dense.weight, w_init)
            if b_init:
                fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.dense.weight)
                bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
                nn.init.uniform_(self.bias, -bound, bound)
            else:
                nn.init.zeros_(self.dense.bias)
        else:
            nn.init.kaiming_normal_(self.dense.weight,a=math.sqrt(5))
            if b_init:
                fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.dense.weight)
                bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
                nn.init.uniform_(self.bias, -bound, bound)
            else:
                nn.init.zeros_(self.dense.bias)

    def forward(self,x):
        """
            Forward propagate
        """
        if self.activation is not None:
            out = self.activation(self.dense(x))
        else:
            out = self.dense(x)
        return out

In [90]:
class ResidualLayerPerceptron(torch.nn.Module):
    def __init__(
        self,
        in_dim     = None,
        out_dim    = None,
        w_init     = False,
        b_init     = False,
        act        = nn.Tanh(),
    ):
        """
            Initialize ResidualLayerPerceptron
        """
        super(ResidualLayerPerceptron,self).__init__()

        if in_dim != out_dim:
            raise ValueError("in_dim of ResBlock should be equal of out_dim, but got in_dim: {}, "
                             "out_dim: {}".format(in_dim, out_dim))

        self.dense      = nn.Linear(in_features=in_dim,
                                    out_features=out_dim,
                                    dtype=torch.float)

        self.activation = act
        # Initialize parameters
        self.init_param(w_init, b_init)

    def init_param(self, w_init, b_init):
        """
            Initialize parameters
        """
        if w_init:
            nn.init.constant_(self.dense.weight, w_init)
            if b_init:
                fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.dense.weight)
                bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
                nn.init.uniform_(self.bias, -bound, bound)
            else:
                nn.init.zeros_(self.dense.bias)
        else:
            nn.init.kaiming_normal_(self.dense.weight,a=math.sqrt(5))
            if b_init:
                fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.dense.weight)
                bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
                nn.init.uniform_(self.bias, -bound, bound)
            else:
                nn.init.zeros_(self.dense.bias)

    def forward(self,x):
        """
            Forward propagate
        """
        out = self.activation(self.dense(x)+x)
        return out

In [91]:
class MLPSequential(torch.nn.Module):
    def __init__(
        self,
        in_dim     = None,
        out_dim    = None,
        layers     = None,
        neurons    = None,
        residual   = False,
        w_init     = False,
        b_init     = False,
        act        = nn.Tanh(),
    ):
        """
            Initialize MLPSequential
        """
        super(MLPSequential,self).__init__()
        if layers < 3:
            raise ValueError("MLPSequential have at least 3 layers, but got layers: {}".format(layers))

        # Define net
        self.net = nn.Sequential()
        self.net.add_module("input", LayerPerceptron(in_dim=in_dim,
                                                           out_dim=neurons,
                                                           w_init=w_init,
                                                           b_init=b_init,
                                                           act=act,
                                                           ))
        for idx in range(layers - 2):
            if residual:
                self.net.add_module(f"res_{idx+1:02d}", ResidualLayerPerceptron(in_dim=neurons,
                                                                                     out_dim=neurons,
                                                                                     w_init=w_init,
                                                                                     b_init=b_init,
                                                                                     act=act,
                                                                                     ))
            else:
                self.net.add_module(f"mlp_{idx+1:02d}", LayerPerceptron(in_dim=neurons,
                                                                        out_dim=neurons,
                                                                        w_init=w_init,
                                                                        b_init=b_init,
                                                                        act=act,
                                                                        ))

        self.net.add_module("output", LayerPerceptron(in_dim=neurons,
                                                      out_dim=out_dim,
                                                      w_init=w_init,
                                                      b_init=b_init,
                                                      act=None,
                                                      ))       


    def forward(self,x):
        """
            Forward propagate
        """
        out = self.net(x)
        return out

In [129]:
class InputScaleNet(torch.nn.Module):
    def __init__(
        self,
        scales     = [],
        centers    = None,
    ):
        """
            Initialize InputScaleNet
        """
        super(InputScaleNet,self).__init__()
        self.scales     = torch.from_numpy(np.array(scales)).type(torch.float)
        if centers is None:
            self.centers = torch.zeros_like(self.scales, dtype=torch.float)
        else:
            self.centers = torch.from_numpy(np.array(centers)).type(torch.float)


    def forward(self,x):
        """
            Forward propagate
        """
        out = torch.mul(x - self.centers, self.scales)
        return out

In [204]:
class MultiScaleMLPSequential(torch.nn.Module):
    def __init__(
        self,
        in_dim     = None,
        out_dim    = None,
        layers     = None,
        neurons    = None,
        residual   = False, # use residual | [True, False]
        w_init     = False, # constant std | (ex 0.1, 0.01)
        b_init     = False, # use bias | [True, False]
        act        = nn.Tanh(),
        subnets    = None,  # subnet(multi scale) number
        amp        = 1.0,   # amplification factor of input
        base_scale = 2.0,   # base scale factor
        in_scale   = None,  # scale factor of input (ex [x,y,t])
        in_center  = None,  # Center position of coordinate translation (ex [x,y,t])
        latent_vec = None,  # latent vector (ex Tensor[shape=(4,16)] )
    ):
        """
            Initialize MultiScaleMLPSequential
        """
        super(MultiScaleMLPSequential,self).__init__()
        self.subnets = subnets
        self.scale_coef = [amp * (base_scale**i) for i in range(self.subnets)]
        self.latent_vec = latent_vec
        
        if self.latent_vec is not None:
            self.num_scenarios = latent_vec.shape[0]
            self.latent_size = latent_vec.shape[1]
            in_dim += self.latent_size
        else:
            self.num_scenarios = 1
            self.latent_size = 0

        # Define MultiScaleMLP
        self.msnet = nn.Sequential()
        for i in range(self.subnets):
            self.msnet.add_module(f"Scale_{i+1}_Net",MLPSequential(in_dim=in_dim,
                                                               out_dim=out_dim,
                                                               layers=layers,
                                                               neurons=neurons,
                                                               residual=residual,
                                                               w_init=w_init,
                                                               b_init=b_init,
                                                               act=act,
                                                               ))
        if in_scale:
            self.in_scale = InputScaleNet(in_scale, in_center)
        else:
            self.in_scale = torch.nn.Identity()


    def forward(self,x):
        """
            Forward propagate
        """
        x = self.in_scale(x)
        if self.latent_vec is not None:
            batch_size = x.shape[0]
            latent_vectors = self.latent_vec.view(self.num_scenarios, 1, self.latent_size).repeat(1,batch_size//self.num_scenarios,1).view(-1,self.latent_size)
            x = torch.concat([x, latent_vectors], axis=1)
        
        out = 0
        for i in range(self.subnets):
            x_s = x * self.scale_coef[i]
            out = out + self.msnet[i](x_s)
        return out

In [205]:
L1 = LayerPerceptron(in_dim=3,
               out_dim=64)
L2 = ResidualLayerPerceptron(in_dim=64,
                            out_dim=64)

In [206]:
input_sample = torch.from_numpy(np.array([0.0, 0.5, 0.3])).type(torch.float)
input_sample

tensor([0.0000, 0.5000, 0.3000])

In [207]:
L1_result = L1(input_sample)
L2_result = L2(L1_result)

print(L1_result)
print(L2_result)

tensor([ 0.1282,  0.0256, -0.2003,  0.3692,  0.3741,  0.0310,  0.0847,  0.2083,
        -0.0165,  0.2475,  0.1940,  0.2290,  0.1718, -0.0896,  0.1503, -0.0067,
         0.1715,  0.0633, -0.1786,  0.1040, -0.2078,  0.2289,  0.1062, -0.0383,
        -0.2619,  0.1027,  0.1535,  0.0044,  0.0018, -0.3223, -0.1883,  0.2138,
         0.0862, -0.1019, -0.1016, -0.2471, -0.0405,  0.0485, -0.0236, -0.0983,
         0.1831, -0.0736, -0.2003,  0.1810, -0.1072,  0.0969,  0.1250, -0.1792,
        -0.0718, -0.0333, -0.3354, -0.1539,  0.4142, -0.2296,  0.2455,  0.2373,
         0.0882, -0.2062,  0.4431,  0.2342, -0.1151, -0.2988,  0.4287, -0.1166],
       grad_fn=<TanhBackward0>)
tensor([ 0.0191, -0.0354, -0.1971,  0.3699,  0.3556,  0.0490, -0.0100,  0.3530,
         0.1180,  0.1009,  0.2671,  0.2121,  0.1149, -0.0849,  0.0886, -0.1250,
        -0.0131,  0.3430, -0.1629,  0.0501, -0.1943,  0.2777,  0.0266, -0.0213,
        -0.2530,  0.1224,  0.3085, -0.0211, -0.1696, -0.2420, -0.1503,  0.2234,
       

In [208]:
net = MLPSequential(
        in_dim     = 3,
        out_dim    = 1,
        layers     = 5,
        neurons    = 64,
        residual   = False,
        w_init     = False,
        b_init     = False,
        act        = nn.Tanh(),
    )

In [209]:
net

MLPSequential(
  (net): Sequential(
    (input): LayerPerceptron(
      (dense): Linear(in_features=3, out_features=64, bias=True)
      (activation): Tanh()
    )
    (mlp_01): LayerPerceptron(
      (dense): Linear(in_features=64, out_features=64, bias=True)
      (activation): Tanh()
    )
    (mlp_02): LayerPerceptron(
      (dense): Linear(in_features=64, out_features=64, bias=True)
      (activation): Tanh()
    )
    (mlp_03): LayerPerceptron(
      (dense): Linear(in_features=64, out_features=64, bias=True)
      (activation): Tanh()
    )
    (output): LayerPerceptron(
      (dense): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)

In [210]:
net(input_sample)

tensor([0.0321], grad_fn=<AddBackward0>)

In [211]:
scale_norm = InputScaleNet(scales=[2.0],
                          centers=[1.0])

In [212]:
scale_norm(input_sample)

tensor([-2.0000, -1.0000, -1.4000])

In [213]:
net = MultiScaleMLPSequential(
    in_dim=3,
    out_dim=1,
    layers=7,
    neurons=32,
    residual=True,
    subnets=3,
    in_scale= [1/5.0,1/20.0,1/100.0],
    in_center=[3.0, 10.0, -15.0],
    latent_vec= torch.from_numpy(np.random.randn(4, 12) / np.sqrt(12)).type(torch.float)
)

In [214]:
net.msnet[0]

MLPSequential(
  (net): Sequential(
    (input): LayerPerceptron(
      (dense): Linear(in_features=15, out_features=32, bias=True)
      (activation): Tanh()
    )
    (res_01): ResidualLayerPerceptron(
      (dense): Linear(in_features=32, out_features=32, bias=True)
      (activation): Tanh()
    )
    (res_02): ResidualLayerPerceptron(
      (dense): Linear(in_features=32, out_features=32, bias=True)
      (activation): Tanh()
    )
    (res_03): ResidualLayerPerceptron(
      (dense): Linear(in_features=32, out_features=32, bias=True)
      (activation): Tanh()
    )
    (res_04): ResidualLayerPerceptron(
      (dense): Linear(in_features=32, out_features=32, bias=True)
      (activation): Tanh()
    )
    (res_05): ResidualLayerPerceptron(
      (dense): Linear(in_features=32, out_features=32, bias=True)
      (activation): Tanh()
    )
    (output): LayerPerceptron(
      (dense): Linear(in_features=32, out_features=1, bias=True)
    )
  )
)

In [215]:
input_sample = torch.rand(size=(16,3), dtype=torch.float)
input_sample.shape

torch.Size([16, 3])

In [216]:
net(input_sample)

tensor([[ 0.3746],
        [ 0.3652],
        [ 0.3747],
        [ 0.3637],
        [-0.0305],
        [-0.0846],
        [-0.0801],
        [-0.0306],
        [ 0.1266],
        [ 0.1427],
        [ 0.1109],
        [ 0.0868],
        [ 0.2716],
        [ 0.3088],
        [ 0.2672],
        [ 0.3007]], grad_fn=<AddBackward0>)